In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset
import json

/mnt/pycharmprojects/cs234_final/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# load model and dataset - dataset needs to be in a specific format
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# get dataset
#train_dataset = load_dataset("imdb", split="train")
dpo_dataset_dict = {
    "prompt": [
        "hello",
        "how are you",
        "What is your name?",
        "What is your name?",
        "Which is the best programming language?",
        "Which is the best programming language?",
        "Which is the best programming language?",
    ],
    "chosen": [
        "hi nice to meet you",
        "I am fine",
        "My name is Mary",
        "My name is Mary",
        "Python",
        "Python",
        "Java",
    ],
    "rejected": [
        "leave me alone",
        "I am not fine",
        "Whats it to you?",
        "I dont have a name",
        "Javascript",
        "C++",
        "C++",
    ],
}
train_dataset = Dataset.from_dict(dpo_dataset_dict)
#print(train_dataset["prompt"][0])

In [8]:
# load trainer
NUM_TRAIN_EPOCHS = 10
OUTPUT_DIR = f"output/epoch={NUM_TRAIN_EPOCHS}"
training_args = DPOConfig(
    beta=0.1,
    # does not automatically save model output
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS
)
trainer = DPOTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
)

# train
results = trainer.train()

/mnt/pycharmprojects/trl/trl/trainer/dpo_trainer.py:363: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/mnt/pycharmprojects/trl/trl/trainer/dpo_trainer.py:376: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/mnt/pycharmprojects/trl/trl/trainer/dpo_trainer.py:411: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
Map: 100%|██████████| 7/7 [00:00<00:00, 508.71 examples/s]
Could not estimate the number of tokens of the input, floating-point operations will not be computed


In [9]:
# save results
with open(f"{OUTPUT_DIR}/results.json", "w") as f:
    json.dump(results.metrics, f)
model.save_pretrained(OUTPUT_DIR)